In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
root.gStyle.SetOptFit(1111)
%jsroot on
%rm output/deltas/*.png

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
sim = 0
Min_Entries = 10
N_files = 1
Do_save = 1

In [3]:
N_hists = 6
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan,root.kTeal]
file_path="input/"
file_names="my-m_ee_Run14AuAu_101st_new_19924_652runs" 
#m_ee_Run14AuAu_101st_new_19924_652runs
#m_ee_Run14AuAu_96th_new_19915_809runs m_ee_Run14AuAu_97th_new_19917_349runs
#m_ee_Run14AuAu_94th_new_19913_757runs m_ee_Run14AuAu_95th_new_19914_1055runs
#m_ee_Run14AuAu_91st_new_19891_1061runs m_ee_Run14AuAu_92nd_new_19900_1065runs m_ee_Run14AuAu_93rd_new_19904_209runs
#m_ee_Run14AuAu_84th_new_19850_337runs m_ee_Run14AuAu_83rd_new_19842_1065runs m_ee_Run14AuAu_85th_new_19858_120runs m_ee_Run14AuAu_86th_new_19862_872runs m_ee_Run14AuAu_87th_new_19865_518runs
hist_select_3D_names = ["DCA12_hist_0","DCA12_hist_1","DCA12_hist_2","DCA12_hist_3","DCA12_hist_4","hist_dca_x","hist_dca_y"]
N_hists = len(hist_select_3D_names)

In [4]:
if sim :
    file_path="../sim/output/Newembed/dca/" 
    file_names="piminus_50M_new_v9"#piminus_50M_v7 piminus_50M_v8 piminus_50M_v9
    N_hists = N_hists - 2

In [5]:
hists_read = []

infile = root.TFile.Open(file_path+file_names+f".root", "read")
for ihist in range(N_hists):
    hists_read.append(infile.Get(hist_select_3D_names[ihist]))
    hists_read[-1].SetDirectory(root.nullptr)
infile.Close()

In [6]:
z_edges = [-12+3*i for i in range(9)]
z_means = [-10.5+3*i for i in range(8)]
z_edges = array('d', z_edges)
z_means = array('d', z_means)
phi_edges = [-1.5 + 0.30*i for i in range(21)]
phi_means = [-1.35 + 0.30*i for i in range(20)]
phi_edges = array('d', phi_edges)
phi_means = array('d', phi_means)
def get_phi_zed_bin(in_value):
    if in_value > 179: in_value -= 180
    zed = int(in_value / 20)
    phi = int((in_value - zed * 20) )
    return zed, phi
def get_phi_value(phi_bin):
    return phi_means[phi_bin] if phi_bin >= 0 and phi_bin < len(phi_means) else 0
def get_zed_value(zed_bin):
    return z_means[zed_bin] if zed_bin >= 0 and zed_bin < len(z_means) else 0

In [7]:
def copycat_hist(hist,seed=0):
    seed += root.gRandom.Integer(10000)
    new_hist = root.TH3D(hist.GetName()+f"{seed}", hist.GetTitle()+f"{seed}", hist.GetNbinsX(), hist.GetXaxis().GetBinLowEdge(1), hist.GetXaxis().GetBinUpEdge(hist.GetNbinsX()),\
        hist.GetNbinsY(), hist.GetYaxis().GetBinLowEdge(1), hist.GetYaxis().GetBinUpEdge(hist.GetNbinsY()),\
        hist.GetNbinsZ(), hist.GetZaxis().GetBinLowEdge(1), hist.GetZaxis().GetBinUpEdge(hist.GetNbinsZ()))
    for ix in range(1, hist.GetNbinsX()+1):
        for iy in range(1, hist.GetNbinsY()+1):
            for iz in range(1, hist.GetNbinsZ()+1):
                new_hist.SetBinContent(ix, iy, iz, hist.GetBinContent(ix, iy, iz))
    return new_hist

In [8]:
ihits = 1
in_hist = hists_read[ihits]
c1 = root.TCanvas("c1", "c1", 1600, 1000)
c1.Divide(3,2)
local_bin_endges = [[1,100],[21,30],[51,60],[25,25],[55,55]]
sigmas = []
for i in range(5):
    if sim: continue
    c1.cd(i+1)
    hist_proj = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj{i}", 66, 101, local_bin_endges[i][0], local_bin_endges[i][1])
    #hist_proj.Draw("")
    hist_proj1 = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj{i}_122", 1, 35, local_bin_endges[i][0], local_bin_endges[i][1])
    hist_proj1.Draw("")
    #hist_proj.Fit("gaus", "Q","",-150,150)
    hist_proj1.Fit("gaus", "Q","",-150,150)
    sigmas.append([hist_proj1.GetFunction("gaus").GetParameter(1), hist_proj1.GetFunction("gaus").GetParameter(2)])

c1.cd(6)
hist_projy = in_hist.ProjectionY(in_hist.GetName()+"hist_projy", 21, 30, 1, 160)
#hist_projy.Draw("")

c1.Draw()

In [9]:
ihits = 1
in_hist = hists_read[ihits]
c1 = root.TCanvas("c1", "c1", 900, 900)
local_bin_endges = [[1,100],[21,30],[51,60],[25,25],[55,55]]
sigmas = []
legends = []
for i in range(1,2):
    hist_proj = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj[i]", 66, 101, local_bin_endges[i][0], local_bin_endges[i][1])
    #hist_proj.Draw("")
    hist_proj1 = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj[i]_1", 1, 35, local_bin_endges[i][0], local_bin_endges[i][1])
    for ibin in range(1, hist_proj1.GetNbinsX()+1):
        hist_proj1.SetBinError(ibin, hist_proj1.GetBinContent(ibin) ** 0.5/1000)
        hist_proj1.SetBinContent(ibin, hist_proj1.GetBinContent(ibin) /1000)
    Format_Hist_total(hist_proj1,"DCA_{T} [#mum]","dN/DCA_{T} [10^{3} per 20 #mum]",0.15,0.15,0.01,0.01,0.05,0.05,21,1,colors[0],3,colors[0],1.2,1.44)
    hist_proj1.GetXaxis().SetNdivisions(505)
    #root.gStyle.SetOptFit(1111)
    hist_proj1.Draw("p")
    #hist_proj.Fit("gaus", "Q","",-150,150)
    hist_proj1.Fit("gaus", "Q","",-100,100-5)
    sigmas.append([hist_proj1.GetFunction("gaus").GetParameter(1), hist_proj1.GetFunction("gaus").GetParameter(2)])
    errors = [hist_proj1.GetFunction("gaus").GetParError(1), hist_proj1.GetFunction("gaus").GetParError(2)]
    hist_proj1.GetFunction("gaus").SetLineColor(colors[3])
    hist_proj1.GetFunction("gaus").SetLineWidth(3)
    legends.append(root.TLegend(0.55, 0.7, 0.95, 0.95))
    Format_Legend(legends[-1], 0.035, 22)
    chi2 = hist_proj1.GetFunction("gaus").GetChisquare()
    ndf = hist_proj1.GetFunction("gaus").GetNDF()
    legends[-1].AddEntry(0, f"chi2/ndf = {chi2:.1f}"+"/"+f"{ndf:.1f}", "")
    legends[-1].AddEntry(0, f"constant = {hist_proj1.GetFunction('gaus').GetParameter(0)*1000:.0f} #pm {hist_proj1.GetFunction('gaus').GetParError(0)*1000:.0f}", "")
    legends[-1].AddEntry(0, f"mean = {sigmas[-1][0]:.2f} #pm {errors[0]:.2f} #mum", "")
    legends[-1].AddEntry(0, f"#sigma = {sigmas[-1][1]:.2f} #pm {errors[1]:.2f} #mum", "")
    legends[-1].Draw()
    legends.append(root.TLegend(0.17, 0.7, 0.5, 0.95,"Au+Au 200 GeV"))
    Format_Legend(legends[-1], 0.04, 22)
    legends[-1].AddEntry(hist_proj1, "h^{+} p_{T} > 1.5 GeV", "p")
    legends[-1].AddEntry(hist_proj1.GetFunction("gaus"), "gaus fit", "l")
    legends[-1].Draw()

hist_projy = in_hist.ProjectionY(in_hist.GetName()+"hist_projy", 21, 30, 1, 160)
#hist_projy.Draw("")

c1.Draw()
#c1.SaveAs("output/dca_hadron_peak_fit.pdf")

In [10]:
# ...existing code...
ihits = 1
in_hist = hists_read[ihits]
c1 = root.TCanvas("c1", "c1", 1300, 900)
local_bin_endges = [[1,100],[21,30],[51,60],[25,25],[55,55]]
sigmas = []
legends = []

# Double-Sided Crystal Ball (Python callable for TF1)
def dscb_eval(x, p):
    # p: [0]=norm, [1]=mean, [2]=sigma, [3]=alphaL, [4]=nL, [5]=alphaR, [6]=nR
    xx = x[0]
    N, mu, sig = p[0], p[1], p[2]
    aL, nL = abs(p[3]), max(1.01, p[4])
    aR, nR = abs(p[5]), max(1.01, p[6])
    if sig <= 0:
        return 0.0
    t = (xx - mu) / sig
    if t < -aL:
        A = (nL/aL)**nL * math.exp(-0.5*aL*aL)
        B = nL/aL - aL
        return N * A * (B - t)**(-nL)
    elif t <= aR:
        return N * math.exp(-0.5*t*t)
    else:
        A = (nR/aR)**nR * math.exp(-0.5*aR*aR)
        B = nR/aR - aR
        return N * A * (B + t)**(-nR)

for i in range(1,2):
    hist_proj = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj[i]", 66, 101, local_bin_endges[i][0], local_bin_endges[i][1])
    #hist_proj.Draw("")
    hist_proj1 = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj[i]_1", 1, 35, local_bin_endges[i][0], local_bin_endges[i][1])

    # scale to 10^3 and set Poisson errors accordingly
    for ibin in range(1, hist_proj1.GetNbinsX()+1):
        hist_proj1.SetBinError(ibin, hist_proj1.GetBinContent(ibin) ** 0.5/1000)
        hist_proj1.SetBinContent(ibin, hist_proj1.GetBinContent(ibin) /1000)

    Format_Hist_total(hist_proj1,"DCA_{T} [#mum]","dN/DCA_{T} [10^{3} per 20 #mum]",0.15,0.15,0.01,0.01,0.05,0.05,21,1,colors[0],3,colors[0],1.2,1.44)
    hist_proj1.GetXaxis().SetNdivisions(505)
    hist_proj1.Draw("p")

    # 1) Initial wide Gaussian to get seed
    gaus_init = root.TF1("gaus_init", "gaus", -180, 180)
    hist_proj1.Fit(gaus_init, "QN", "", -120, 120)
    init_mean  = gaus_init.GetParameter(1)
    init_sigma = gaus_init.GetParameter(2)

    # 2) Core Gaussian within ±1.5σ
    core_low  = init_mean - 1.5*init_sigma
    core_high = init_mean + 1.5*init_sigma
    gaus_core = root.TF1("gaus_core", "gaus", core_low, core_high)
    gaus_core.SetParameters(gaus_init.GetParameter(0), init_mean, init_sigma)
    hist_proj1.Fit(gaus_core, "Q", "", core_low, core_high)

    core_mean  = gaus_core.GetParameter(1)
    core_sigma = gaus_core.GetParameter(2)
    core_mean_err  = gaus_core.GetParError(1)
    core_sigma_err = gaus_core.GetParError(2)

    # 3) Double-Sided Crystal Ball fit over wide range, constrained near core mean/sigma
    dscb = root.TF1("dscb", dscb_eval, -150, 150, 7)
    dscb.SetParNames("Norm","Mean","#sigma","#alpha_{L}","n_{L}","#alpha_{R}","n_{R}")
    dscb.SetParameters(hist_proj1.GetMaximum(), core_mean, core_sigma, 1.5, 3.0, 2.0, 3.0)

    # Constrain core around Gaussian core
    dscb.SetParLimits(1, core_mean - 0.5*core_sigma, core_mean + 0.5*core_sigma)         # Mean
    dscb.SetParLimits(2, max(0.3*core_sigma, 2.0), 2.0*core_sigma)                        # Sigma > few microns
    # Tail parameters reasonable ranges
    dscb.SetParLimits(3, 0.3, 5.0)   # alphaL
    dscb.SetParLimits(4, 1.05, 10.)  # nL
    dscb.SetParLimits(5, 0.3, 5.0)   # alphaR
    dscb.SetParLimits(6, 1.05, 10.)  # nR

    hist_proj1.Fit(dscb, "Q", "", -180, 180)

    # Collect parameters
    mean  = dscb.GetParameter(1)
    sigma = dscb.GetParameter(2)
    sigmas.append([mean, sigma])
    errors = [dscb.GetParError(1), dscb.GetParError(2)]

    # Style fits
    dscb.SetLineColor(colors[3])
    dscb.SetLineWidth(3)
    gaus_core.SetLineColor(colors[2])
    gaus_core.SetLineStyle(2)
    gaus_core.SetLineWidth(3)
    gaus_core.Draw("same")

    # Legend
    legends.append(root.TLegend(0.5, 0.54, 0.99, 0.97))
    Format_Legend(legends[-1], 0.035, 22)
    chi2 = dscb.GetChisquare()
    ndf  = dscb.GetNDF()
    legends[-1].AddEntry(0, f"#chi^{{2}}/ndf = {chi2:.1f}/{ndf:.0f}", "")
    legends[-1].AddEntry(0, f"gauss:", "")
    legends[-1].AddEntry(0, f"#mu = {core_mean:.1f} #pm {core_mean_err:.1f}, #sigma = {core_sigma:.0f} #pm {core_sigma_err:.0f}", "")
    legends[-1].AddEntry(0, f"DSCB:", "")
    legends[-1].AddEntry(0, f"#mu = {mean:.1f} #pm {errors[0]:.1f}, #sigma = {sigma:.0f} #pm {errors[1]:.0f} #mum", "")
    legends[-1].AddEntry(0, f"#alpha_{{L}} = {dscb.GetParameter(3):.0f} #pm {dscb.GetParError(3):.0f}, n_{{L}} = {dscb.GetParameter(4):.0f} #pm {dscb.GetParError(4):.0f}", "")
    legends[-1].AddEntry(0, f"#alpha_{{R}} = {dscb.GetParameter(5):.0f} #pm {dscb.GetParError(5):.0f}, n_{{R}} = {dscb.GetParameter(6):.0f} #pm {dscb.GetParError(6):.0f}", "")
    legends[-1].Draw()

    legends.append(root.TLegend(0.16, 0.6, 0.55, 0.95,"Au+Au 200 GeV"))
    Format_Legend(legends[-1], 0.04, 22)
    legends[-1].AddEntry(hist_proj1, "h^{+} p_{T} > 1.5 GeV", "p")
    legends[-1].AddEntry(dscb, "Double-sided Crystal Ball", "l")
    legends[-1].AddEntry(gaus_core, "Gaussian core (#pm1.5#sigma)", "l")
    legends[-1].Draw()

hist_projy = in_hist.ProjectionY(in_hist.GetName()+"hist_projy", 21, 30, 1, 160)
#hist_projy.Draw("")

c1.Draw()
#c1.SaveAs("output/dca_hadron_peak_dscb_core.pdf")
# ...existing code...

In [11]:
print("Sigma values for the histograms:")
for i, sigma in enumerate(sigmas):
    print(f"Histogram [i+1]: [sigma[0]:.3f], [sigma[1]:.3f]")

Sigma values for the histograms:
Histogram [i+1]: [sigma[0]:.3f], [sigma[1]:.3f]


In [12]:
ihits = 1
pt = 15
in_hist = copycat_hist(hists_read[ihits])
for ibinx in range(1,1+in_hist.GetNbinsX()):
    for ibiny in range(1,1+in_hist.GetNbinsY()):
        for ibinz in range(1,1+in_hist.GetNbinsZ()):
            if ibiny<50+pt and ibiny>50-pt:
                in_hist.SetBinContent(ibinx, ibiny, ibinz, 0)
hist_proj = in_hist.Project3D("xz")

c1 = root.TCanvas("c1", "c1", 800, 600)
profile_x = hist_proj.ProfileX(hist_proj.GetName()+"profileX")
profile_x.Rebin(1)
#profile_x.Fit(fit_func, "Q", "", 50, 60)
profile_x.Draw("")
c1.Draw()


In [13]:
if False :
    x_offsets = array("d")
    x_abscissas = array("d")
    x_offsets_errors = array("d")
    x_arrays = []
    for ibinx in range(profile_x.GetNbinsX()):
        x_offsets.append(profile_x.GetBinContent(ibinx+1))
        x_offsets_errors.append(profile_x.GetBinError(ibinx+1))
        x_abscissas.append(profile_x.GetBinCenter(ibinx+1))
    for i, (xx, yy, zz) in enumerate(zip(x_abscissas, x_offsets, x_offsets_errors)):
        z_bin, phi_bin = get_phi_zed_bin(xx)
        phi = get_phi_value(phi_bin)
        if z_bin not in x_arrays:
            x_arrays[z_bin] = [array('d'),array('d'),array('d'),array('d')]
        x_arrays[z_bin][0].append((yy))
        x_arrays[z_bin][1].append((phi))
        x_arrays[z_bin][2].append((zz))
        x_arrays[z_bin][3].append(0.15)
    
    print("x_arrays", x_arrays)

In [14]:
if True :
    x_offsets = array("d")
    x_abscissas = array("d")
    x_offsets_errors = array("d")
    x_arrays = {}
    all_projections = []
    for ibinx in range(profile_x.GetNbinsX()):
        local_mean = in_hist.GetBinContent(ibinx+1)
        local_projection = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj_z_{ibinx}", 1, 99, ibinx+1, ibinx+1)
        local_projection.Fit("gaus", "Q","",local_mean-120,local_mean+120)
        all_projections.append(local_projection)
        if local_projection.GetEntries() > 50 and local_projection.GetFunction("gaus"):
            c1 = root.TCanvas("c1", "c1", 800, 600)
            mean = local_projection.GetFunction("gaus").GetParameter(1)
            sigma = local_projection.GetFunction("gaus").GetParameter(2)
            local_projection.Fit("gaus", "Q","",mean-1.5*sigma,local_mean+1.5*sigma)
            x_offsets.append(local_projection.GetFunction("gaus").GetParameter(1))
            x_offsets_errors.append(2000./local_projection.GetFunction("gaus").Integral(-120, 120) **0.5 )
            local_projection.Draw("")
            c1.SaveAs(f"output/deltas/fit_zed_{ibinx}.png")
        else:
            x_offsets.append(0)
            x_offsets_errors.append(0)
        x_abscissas.append(profile_x.GetBinCenter(ibinx+1))
    for i, (xx, yy, zz) in enumerate(zip(x_abscissas, x_offsets, x_offsets_errors)):
        z_bin, phi_bin = get_phi_zed_bin(xx)
        phi = get_phi_value(phi_bin)
        if z_bin not in x_arrays:
            x_arrays[z_bin] = [array('d'),array('d'),array('d'),array('d')]
        x_arrays[z_bin][0].append((yy))
        x_arrays[z_bin][1].append((phi))
        x_arrays[z_bin][2].append((zz))
        x_arrays[z_bin][3].append(0.15)
    
    print("x_arrays", x_arrays)

x_arrays {0: [array('d', [0.0, 0.0, 7.146436846031433, -8.036290052467413, -11.000966196080435, 5.588164545701819, 3.413994092259878, 7.320200268698708, -17.13092640094559, 0.0, 0.0, -12.132343443767345, -8.990327967339041, -9.950708282374386, -10.119919210156608, -1.1961142754527123, -9.744462736767897, 8.449504754829313, 0.0, 0.0]), array('d', [-1.35, -1.05, -0.7500000000000001, -0.4500000000000002, -0.15000000000000013, 0.1499999999999999, 0.44999999999999973, 0.75, 1.0499999999999998, 1.3499999999999996, 1.65, 1.9499999999999997, 2.2499999999999996, 2.55, 2.85, 3.15, 3.4499999999999997, 3.7499999999999996, 4.049999999999999, 4.35]), array('d', [0.0, 0.0, 36.95684593190718, 11.494833880385075, 10.543842309045678, 10.576433671075222, 6.808113840997436, 8.26017748434817, 17.649989325905345, 0.0, 0.0, 44.762461699699905, 18.098268667926405, 10.833377069280168, 8.244304860172031, 5.257407154730693, 7.007980710320626, 18.415827679431743, 0.0, 0.0]), array('d', [0.15, 0.15, 0.15, 0.15, 0.

In [15]:
if False:
    for iz in x_arrays:
        arr = x_arrays[iz][0]
        arr_errors = x_arrays[iz][2]
        n = len(arr)
        # Find indices of valid (non -200) entries in each region
        valid_indices_low = [i for i, v in enumerate(arr) if v != -200 and i < 10]
        valid_indices_high = [i for i, v in enumerate(arr) if v != -200 and i >= 10]
        for i in range(n):
            if arr[i] == -200:
                if i < 10 and valid_indices_low:
                    closest = min(valid_indices_low, key=lambda x: abs(x - i))
                    arr[i] = arr[closest]
                    arr_errors[i] = arr_errors[closest]
                elif i >= 10 and valid_indices_high:
                    closest = min(valid_indices_high, key=lambda x: abs(x - i))
                    arr[i] = arr[closest]
                    arr_errors[i] = arr_errors[closest]

In [16]:
c3 = root.TCanvas("c3", "c3", 1600, 1600)
c3.Divide(2, 4)
graphs_x = []
graphs_y = []
for iz in range(len(z_means)):
    c3.cd(iz+1)
    if iz not in x_arrays: continue
    graph_x = root.TGraphErrors(len(x_arrays[iz][0]), x_arrays[iz][1], x_arrays[iz][0], x_arrays[iz][3], x_arrays[iz][2])
    graph_x.SetTitle(f"Z={get_zed_value(iz)}")
    graph_x.GetXaxis().SetTitle("Phi")
    graph_x.GetYaxis().SetTitle("DCA X/Y")
    graph_x.SetMarkerStyle(20)
    graph_x.SetMarkerColor(colors[iz])
    graphs_x.append(graph_x)
    graph_x.SetMaximum(50)
    graph_x.SetMinimum(-50)
    graph_x.Draw("AP")
c3.Draw()

In [17]:
# Draw DCA vs z for each phi bin
c_phi = root.TCanvas("c_phi", "DCA vs z for each phi bin", 1600, 1600)
c_phi.Divide(5, 4)  # 20 phi bins

graphs_phi = []
for iphi in range(len(phi_means)):
    c_phi.cd(iphi+1)
    z_vals = array('d')
    dca_vals = array('d')
    dca_errs = array('d')
    z_errs = array('d')
    for iz in range(len(z_means)):
        if iz not in x_arrays:
            continue
        # Find the index of this phi bin in the current z bin
        try:
            phi_indices = [i for i, v in enumerate(x_arrays[iz][1]) if abs(v - phi_means[iphi]) < 1e-6]
            if not phi_indices:
                continue
            idx = phi_indices[0]
            z_vals.append(z_means[iz])
            dca_vals.append(x_arrays[iz][0][idx])#*math.sin(idx+math.pi/2)
            dca_errs.append(x_arrays[iz][2][idx])
            z_errs.append(1.5)  # or use x_arrays[iz][3][idx] if available
        except Exception:
            continue
    if len(z_vals) > 0:
        graph = root.TGraphErrors(len(z_vals), z_vals, dca_vals, z_errs, dca_errs)
        graph.SetTitle(f"Phi={phi_means[iphi]:.2f}")
        graph.GetXaxis().SetTitle("z")
        graph.GetYaxis().SetTitle("DCA")
        graph.SetMarkerStyle(20)
        graph.SetMarkerColor(colors[iphi % len(colors)])
        graphs_phi.append(graph)
        graph.SetMaximum(50)
        graph.SetMinimum(-50)
        graph.Draw("AP")
c_phi.Draw()

In [18]:
print(phi_means)
if True:
    phi_means = phi_means[2:-2]
    phi_means.pop(8)
    phi_means.pop(7)


array('d', [-1.35, -1.05, -0.7500000000000001, -0.4500000000000002, -0.15000000000000013, 0.1499999999999999, 0.44999999999999973, 0.75, 1.0499999999999998, 1.3499999999999996, 1.65, 1.9499999999999997, 2.2499999999999996, 2.55, 2.85, 3.15, 3.4499999999999997, 3.7499999999999996, 4.049999999999999, 4.35])


In [ ]:
# Draw DCA vs z for each phi bin
c_phi = root.TCanvas("c_phi", "DCA vs z for each phi bin", 1600, 1200)
c_phi.Divide(4, 3)  # 20 phi bins

graphs_phi = []
pol1 = []
for iphi in range(len(phi_means)-2):
    c_phi.cd(iphi+1)
    z_vals = array('d')
    dca_vals = array('d')
    dca_errs = array('d')
    z_errs = array('d')
    for iz in range(len(z_means)):
        if iz not in x_arrays:
            continue
        # Find the index of this phi bin in the current z bin
        try:
            phi_indices = [i for i, v in enumerate(x_arrays[iz][1]) if abs(v - phi_means[iphi]) < 1e-6]
            if not phi_indices:
                continue
            idx = phi_indices[0]
            z_vals.append(z_means[iz])
            dca_vals.append(x_arrays[iz][0][idx])#*math.sin(idx+math.pi/2)
            dca_errs.append(x_arrays[iz][2][idx])
            z_errs.append(1.5)  # or use x_arrays[iz][3][idx] if available
        except Exception:
            continue
    if len(z_vals) > 0:
        graph = root.TGraphErrors(len(z_vals), z_vals, dca_vals, z_errs, dca_errs)
        graph.SetTitle(f"#LT#phi#GT={phi_means[iphi]:.2f}")
        graph.GetXaxis().SetTitle("z, cm")
        graph.GetYaxis().SetTitle("#LTDCA_{T}#GT, #mum")
        graph.SetMarkerStyle(20)
        graph.SetMarkerColor(colors[iphi % len(colors)])
        graphs_phi.append(graph)
        graph.SetMaximum(50)
        graph.SetMinimum(-50)
        pol1.append(root.TF1(f"pol1_phi_{iphi}", "pol1", -12, 0))
        graph.Fit(pol1[-1], "QR","",-12,0)
        pol1.append(root.TF1(f"pol1_phi_{iphi+100}", "pol1", 0, 12))
        graph.Fit(pol1[-1], "QR+","",0,12)
        graph.Draw("AP")

        root.gPad.SetBottomMargin(0.12)
        root.gPad.SetLeftMargin(0.12)

        graph.GetXaxis().SetTitleSize(0.05)
        graph.GetXaxis().SetLabelSize(0.05)
        graph.GetXaxis().SetTitleOffset(1.)
        graph.GetYaxis().SetTitleSize(0.05)
        graph.GetYaxis().SetLabelSize(0.05)
        graph.GetYaxis().SetTitleOffset(1.2)
        graph.GetYaxis().CenterTitle()
        graph.GetXaxis().CenterTitle()

c_phi.Draw()
c_phi.SaveAs("output/dca_vs_z_phi_bins_corr.png")

In [20]:
# Example for plotting means in 3D for ilayer=0
z_cut = [-20, 0]
phi_cut = [-1.5, 1.5]
g3d = root.TGraph2D()
point = 0
for ized, z in enumerate(z_means):
    if z < z_cut[0] or z > z_cut[1]:
        continue
    n_phi = len(x_arrays[ized][0])
    for iphi in range(n_phi):
        phi = x_arrays[ized][1][iphi]
        if phi < phi_cut[0] or phi > phi_cut[1]:
            continue
        mean = x_arrays[ized][0][iphi]
        if mean > -1 and mean<1:
            continue
        g3d.SetPoint(point, phi, z_means[ized], mean)
        point += 1
# Define the fit function: ([0]+[1]*zed)*sin(phi)+([2]+[3]*zed)*cos(phi)+[4]
fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x)+([2]+[3]*y)/10000*cos(x)+[4]", -2, 4, z_cut[0], z_cut[1])
fit_func.SetParameters(0, 0, 0, 0, 0)  # You can tune these

c3d = root.TCanvas("c3d", "3D Mean", 800, 600)


g3d.SetTitle("Mean vs #phi and z;#phi;z;Mean")
g3d.SetMarkerStyle(20)
g3d.SetMarkerColor(root.kBlue)
# Fit the 3D graph
g3d.Fit(fit_func, "Q")  # "Q" for quiet

# Draw the points and the fit surface
g3d.Draw("P0 surf1")
fit_func.Draw("same surf1")

c3d.Draw()
#params = fit_func.GetParameters()
#print("Fit parameters:", params[0], params[1], params[2], params[3])

In [21]:
if False:
    for iz in x_arrays:
        arr = x_arrays[iz][0]
        n = len(arr)
        # Find indices of valid (non -200) entries
        valid_indices = [i for i, v in enumerate(arr) if v != -200]
        for i in range(n):
            if arr[i] == -200:
                # Find closest valid index
                if valid_indices:
                    closest = min(valid_indices, key=lambda x: abs(x - i))
                    arr[i] = arr[closest]

In [22]:
if True:
    print(f"const double offsets_dca[{len(x_arrays)}][{len(x_arrays[0][0])}] =" + " {")
    for iz in range(len(x_arrays)):
        if iz > 0: print(",")
        print("    {", end="")
        for i in range(len(x_arrays[iz][0])):
            if i > 0: print(", ", end="")
            print(f"{x_arrays[iz][0][i]/10000:.3g}", end="")
        print("}", end="")
    print("\n};")

const double offsets_dca[8][20] = {
    {0, 0, 0.000715, -0.000804, -0.0011, 0.000559, 0.000341, 0.000732, -0.00171, 0, 0, -0.00121, -0.000899, -0.000995, -0.00101, -0.00012, -0.000974, 0.000845, 0, 0},
    {0, 0, 0.000448, -0.0014, -0.000911, 0.000498, 2.41e-05, 0.000722, -0.00206, 0, 0, -0.00176, -0.00116, -0.0014, -0.000566, -0.000514, -0.000597, 0.000933, 0, 0},
    {0, 0, 0.000329, -0.00151, -0.000242, -0.000259, 0.000185, 0.000849, -0.00123, 0, 0, -0.00208, -0.00295, -0.00228, -0.000348, -0.00101, -0.000802, 0.00118, 0, 0},
    {0, 0, -0.000384, -0.0011, 0.000202, -0.000276, 0.000128, 0.000833, -0.00173, 0, 0, -0.00188, -0.00184, -0.00209, -0.000537, -0.000927, -0.00127, 0.00161, 0, 0},
    {0, 0, 0.000638, -0.000486, 0.000637, 0.000278, -0.000205, 0.000384, -4.88e-05, 0, 0, 4.18e-05, -0.000831, -0.00146, -0.000328, -0.000488, -0.0006, 0.00214, 0, 0},
    {0, 0, -0.000626, -0.000872, 5.64e-05, 0.000622, -0.000211, 0.000271, 4.11e-05, 0, 0, -0.000555, -0.00249, -0.00146, -0.000302

In [23]:
if True:
    print(f"const double offsets_dca_y[{len(x_arrays)}][2] =" + " {")
    for iz in range(len(x_arrays)):
        if iz > 0: print(",")
        print("    {"+f"{(x_arrays[iz][0][4]+x_arrays[iz][0][5])/10000:.3g}"+f", {x_arrays[iz][0][15]/10000:.3g}", end="")
        print("}", end="")
    print("\n};")

const double offsets_dca_y[8][2] = {
    {-0.000541, -0.00012},
    {-0.000413, -0.000514},
    {-0.0005, -0.00101},
    {-7.4e-05, -0.000927},
    {0.000915, -0.000488},
    {0.000679, 0.000167},
    {-0.000472, 2.63e-05},
    {-0.000453, -7.25e-05}
};


## Formula for Alignment Fit

The function used for fitting the mean shift as a function of azimuthal angle (φ) and longitudinal position (z) is:

<div align="center">

Δφ(z, φ) = ([0] + [1]·z)/10000 · sin(φ) + ([2] + [3]·z)/10000 · cos(φ) + [4]/1000

</div>

**where:**
- [0], [1], [2], [3], [4] are fit parameters extracted from the data,
- z is the longitudinal coordinate (e.g., along the beam axis),
- φ is the azimuthal angle.

---

### Why use this function for a 3D histogram fit?

- **Physical Motivation:**  
  Misalignments of detector layers often manifest as sinusoidal modulations in the measured hit positions as a function of φ, with possible linear dependence on z due to tilts or twists.

- **Parameter Extraction:**  
  Fitting this function to the 3D histogram of mean residuals (mean shift vs. φ and z) allows extraction of alignment parameters:
    - The coefficients of sin(φ) and cos(φ) (and their z-dependence) correspond to translations and tilts of the detector layer in the transverse plane.
    - The constant term can represent a global offset.

- **Usage:**  
  After fitting, the obtained parameters can be used to correct (align) the detector geometry, improving tracking and vertexing performance.